In [ ]:
#조선왕조실록 rag
import os
from openai import AzureOpenAI

AZURE_OPENAI_ENDPOINT = "https://5team-history-azureai-services.openai.azure.com/"
AZURE_OPENAI_DEPLOYMENT = "jeoson-rag"
AZURE_OPENAI_API_VERSION = "2025-01-01-preview"
AZURE_OPENAI_API_KEY = "api.key"

# 클라이언트 생성
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
)

system_message = (
    "너는 RAG 시스템과 연동된 조선시대 한국사 전문가야. 사용자의 질문에 대해 벡터 검색된 사료 청크를 바탕으로만 답변하고, 반드시 해당 청크에서 발췌한 문장을 명시하거나 출처를 표기해야 해. 사료에 존재하지 않는 정보는 추측하지 말고 '해당 정보는 실록에서 확인되지 않습니다'라고 말해줘"
)

def extract_keywords(query):
    prompt = f"다음 문장에서 핵심 키워드 3~5개만 '키워드1, 키워드2, ...' 형식으로 추출해줘:\n{query}\n>"
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=100,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )
    keywords = response.choices[0].message.content.strip()
    return [kw.strip() for kw in keywords.split(",")]

def evaluate_answer(keywords, answer):
    eval_prompt = (
        f"다음 키워드: {', '.join(keywords)}\n아래 답변이 키워드를 잘 반영하고 있는지 "
        f"조선시대 한국사 전문가 AI 시각으로 평가해줘:\n{answer}"
    )
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=200,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": eval_prompt},
        ],
    )
    return response.choices[0].message.content.strip()

def get_answer(query):
    rag_prompt = (
        f"{query}\n(다음 내용은 조선왕조실록을 참조하여 고증한 내용으로 작성되어야 합니다.)"
    )
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=1024,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": rag_prompt},
        ],
    )
    return response.choices[0].message.content.strip()

if __name__ == "__main__":
    query = "조선 중기의 양반들의 삶"

    # 1. 키워드 추출
    keywords = extract_keywords(query)

    # 2. 질문에 대한 답변 생성
    answer = get_answer(query)

    # 3. AI가 답변과 키워드 매칭 평가
    ai_eval = evaluate_answer(keywords, answer)

    # 4. JSON 결과 구성 및 출력
    result_json = {
        "keywords": keywords,
        "ai_eval": ai_eval,
        "answer": answer,
    }
    print(result_json)


{'keywords': ['조선 중기', '양반', '삶'], 'ai_eval': '전문가 평가:\n\n1. **키워드 반영 여부**  \n질문에서 제시된 키워드인 ‘조선 중기’, ‘양반’, ‘삶’이 모두 충실하게 반영되어 있습니다. 답변은 조선 중기라는 시기적 범위와 양반 계층에 초점을 두고, 경제 생활, 사회적 특권, 일상/교육, 정치 참여 등 삶의 다양한 측면을 실록의 구체적 기사와 연결해 설명하고 있습니다.\n\n2. **사료(실록) 인용 및 출처 표기 적절성**  \n각 항목마다 실록의 특정 기사(권, 연도, 날짜)를 명확히 표기하고 해당 기사에서 발췌한 내용(직접 인용 또는 요약)을 제시하였으므로, 사료에 근거한 설명이라는 점이 명확하게 드러납니다.  \n예)', 'answer': '조선 중기의 양반들의 삶에 대해 조선왕조실록에 근거하여 말씀드리겠습니다.\n\n1. 경제 생활  \n조선 중기 양반들은 토지를 소유하여 농민에게 소작을 주고 그 수확의 일부를 수취하는 것이 일반적이었습니다. 《중종실록》 26권, 중종 12년 9월 3일자 기사에는 "양반들이 전답을 많이 소유하여 백성들의 형편이 곤란하다"라는 언급이 있어, 양반의 토지 소유와 농민과의 관계가 드러납니다.\n\n2. 사회적 특권  \n양반은 과거제(科擧制)를 통해 관직에 나아갈 수 있는 신분이었으며, 법적으로도 여러 특권을 누렸습니다. 《선조실록》 69권, 선조 50년 1월 10일자 기사에서는 "양반 자제만 과거에 응시할 수 있으니, 신분에 따라 관직 진출이 제한된다"고 기록되어 있습니다.\n\n3. 일상 및 교육  \n양반 가문에서는 자제들에게 유학(儒學)을 가르치고, 효(孝)와 예(禮)를 중시하였습니다. 《광해군일기》 93권, 광해군 7년 12월 19일자 기사에는 "양반 집안에서는 아침저녁으로 글 읽는 소리가 끊이지 않는다"고 언급되어 있습니다.\n\n4. 정치 참여와 향촌 운영  \n양반들은 지방의 향약(鄕約) 등 자치 조직을 운영하며 향촌 사회를 주도하였습니다. 《중종실록》 75권, 

In [ ]:
#조선왕조실록 시놉시스rag
import os
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
from datetime import datetime
import re

AZURE_SEARCH_ENDPOINT = "https://5team-ai-search.search.windows.net"
AZURE_SEARCH_INDEX_NAME = "rag-ksat"
AZURE_SEARCH_API_KEY = "apikey"

AZURE_OPENAI_ENDPOINT = "https://5team-history-azureai-services.openai.azure.com/"
AZURE_OPENAI_DEPLOYMENT = "jeoson-rag"
AZURE_OPENAI_API_VERSION = "2025-01-01-preview"
AZURE_OPENAI_API_KEY = "apikey"

# 클라이언트 초기화
search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=AzureKeyCredential(AZURE_SEARCH_API_KEY)
)

openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
)

def print_header(title, width=80):
    """헤더 출력"""
    print("\n" + "="*width)
    print(f"🎯 {title}")
    print("="*width)

def print_section(title, width=60):
    """섹션 출력"""
    print(f"\n{'='*10} {title} {'='*10}")

def print_subsection(title):
    """하위 섹션 출력"""
    print(f"\n📌 {title}")
    print("-" * 50)

def extract_keywords_from_query(query, top_k=10, max_final_docs=10):
    """
    AI를 이용해 쿼리에서 검색 키워드 자동 추출
    
    Args:
        query (str): 사용자 입력 쿼리
        top_k (int): 키워드당 검색할 문서 수
        max_final_docs (int): 최종 선별할 문서 수
        
    Returns:
        tuple: (추출된 키워드 리스트, 키워드 추출 상세 정보)
    """
    print_subsection("AI 기반 키워드 자동 추출")
    print(f"🔍 입력 쿼리: {query}")
    
    keyword_extraction_prompt = f"""
사용자 쿼리: "{query}"

위 쿼리를 분석하여 Azure Search에서 관련 문서를 찾기 위한 최적의 검색 키워드들을 추출해주세요.

추출 기준:
1. 쿼리의 핵심 주제와 관련된 키워드
2. 역사적 인물명, 사건명, 제도명
3. 시대적 배경 키워드  
4. 관련 개념어 및 전문용어
5. 유사한 의미의 동의어들
6. 검색에 유용한 단일어 우선 (2-3글자 단어들)

응답 형식:
핵심키워드: [가장 중요한 키워드 5개를 콤마로 구분]
확장키워드: [관련된 추가 키워드 10개를 콤마로 구분] 
동의어키워드: [동의어나 유사어 5개를 콤마로 구분]
인물키워드: [관련 인물명 5개를 콤마로 구분 - 있다면]
시대키워드: [시대/왕조 관련 3개를 콤마로 구분 - 있다면]

추출 이유: [키워드 선택 이유를 2-3줄로 설명]
"""
    
    try:
        print("🤖 AI 키워드 추출 중...")
        
        response = openai_client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,
            max_tokens=800,
            temperature=0.3,
            messages=[
                {
                    "role": "system", 
                    "content": "한국사 전문가이자 검색 키워드 추출 전문가입니다. 사용자 쿼리에서 문서 검색에 가장 효과적인 키워드들을 추출합니다."
                },
                {
                    "role": "user", 
                    "content": keyword_extraction_prompt
                }
            ]
        )
        
        extraction_result = response.choices[0].message.content.strip()
        print("✅ AI 키워드 추출 완료")
        
        # 키워드 파싱
        keywords = []
        extraction_info = {
            'core': [],
            'extended': [], 
            'synonyms': [],
            'persons': [],
            'periods': [],
            'reasoning': ''
        }
        
        lines = extraction_result.split('\n')
        for line in lines:
            line = line.strip()
            if line.startswith('핵심키워드:'):
                core_kw = line.replace('핵심키워드:', '').strip()
                extraction_info['core'] = [kw.strip() for kw in core_kw.split(',') if kw.strip()]
                keywords.extend(extraction_info['core'])
                
            elif line.startswith('확장키워드:'):
                ext_kw = line.replace('확장키워드:', '').strip()
                extraction_info['extended'] = [kw.strip() for kw in ext_kw.split(',') if kw.strip()]
                keywords.extend(extraction_info['extended'])
                
            elif line.startswith('동의어키워드:'):
                syn_kw = line.replace('동의어키워드:', '').strip()
                extraction_info['synonyms'] = [kw.strip() for kw in syn_kw.split(',') if kw.strip()]
                keywords.extend(extraction_info['synonyms'])
                
            elif line.startswith('인물키워드:'):
                person_kw = line.replace('인물키워드:', '').strip()
                extraction_info['persons'] = [kw.strip() for kw in person_kw.split(',') if kw.strip()]
                keywords.extend(extraction_info['persons'])
                
            elif line.startswith('시대키워드:'):
                period_kw = line.replace('시대키워드:', '').strip()
                extraction_info['periods'] = [kw.strip() for kw in period_kw.split(',') if kw.strip()]
                keywords.extend(extraction_info['periods'])
                
            elif line.startswith('추출 이유:'):
                extraction_info['reasoning'] = line.replace('추출 이유:', '').strip()
        
        # 중복 제거 및 정리
        unique_keywords = []
        seen = set()
        for kw in keywords:
            if kw and kw not in seen and len(kw.strip()) >= 2:
                unique_keywords.append(kw.strip())
                seen.add(kw)
        
        print(f"📊 추출된 키워드 통계:")
        print(f"   - 핵심키워드: {len(extraction_info['core'])}개")
        print(f"   - 확장키워드: {len(extraction_info['extended'])}개") 
        print(f"   - 동의어키워드: {len(extraction_info['synonyms'])}개")
        print(f"   - 인물키워드: {len(extraction_info['persons'])}개")
        print(f"   - 시대키워드: {len(extraction_info['periods'])}개")
        print(f"   - 총 유효 키워드: {len(unique_keywords)}개")
        
        print(f"\n🔑 최종 키워드 목록:")
        for i, kw in enumerate(unique_keywords[:20], 1):  # 상위 20개만 출력
            print(f"   {i:2d}. {kw}")
        
        if len(unique_keywords) > 20:
            print(f"   ... 외 {len(unique_keywords) - 20}개")
            
        print(f"\n💡 추출 이유: {extraction_info['reasoning']}")
        
        return unique_keywords, extraction_info
        
    except Exception as e:
        print(f"⚠️ 키워드 추출 오류: {e}")
        # 실패 시 쿼리에서 간단한 키워드 추출
        fallback_keywords = []
        # 한글 2글자 이상 단어 추출
        korean_words = re.findall(r'[가-힣]{2,}', query)
        fallback_keywords.extend(korean_words[:10])
        
        print(f"🔄 대안 키워드 사용: {fallback_keywords}")
        return fallback_keywords, {'reasoning': '자동 추출 실패로 쿼리 단어 사용'}

def search_documents_with_keywords(keywords, top_k=10, max_final_docs=10):
    """
    추출된 키워드로 문서 검색
    
    Args:
        keywords (list): 검색 키워드 리스트
        top_k (int): 키워드당 검색할 문서 수  
        max_final_docs (int): 최종 선별할 문서 수
        
    Returns:
        list: 검색된 관련 문서들
    """
    print_subsection(f"키워드 기반 문서 검색")
    print(f"🔍 검색 키워드: {len(keywords)}개")
    print(f"📊 검색 설정: 키워드당 {top_k}개, 최종 {max_final_docs}개")
    
    all_docs = []
    search_stats = {"total_searched": 0, "found_docs": 0, "total_results": 0}
    
    print(f"\n{'키워드':<15} | {'검색결과':<8} | {'관련문서':<8} | {'최고점수':<10} | {'상태'}")
    print("-" * 70)
    
    for keyword in keywords:
        try:
            results = search_client.search(
                search_text=keyword,
                top=top_k,
                search_mode="any"
            )
            
            search_stats["total_searched"] += 1
            found_count = 0
            max_score = 0
            
            for result in results:
                chunk = result.get('chunk', '')
                score = result.get('@search.score', 0)
                search_stats["total_results"] += 1
                max_score = max(max_score, score)
                
                if chunk and len(chunk.strip()) > 50:  # 의미있는 내용만
                    clean_preview = ' '.join(chunk.replace('\n', ' ').split())
                    preview_text = clean_preview[:100] + "..." if len(clean_preview) > 100 else clean_preview
                    
                    all_docs.append({
                        'keyword': keyword,
                        'chunk': chunk,
                        'score': score,
                        'preview': preview_text
                    })
                    found_count += 1
            
            if found_count > 0:
                search_stats["found_docs"] += 1
                status = "✅ 발견"
            else:
                status = "❌ 없음"
                
            print(f"{keyword:<15} | {search_stats['total_results']:>8} | {found_count:>8} | {max_score:>10.2f} | {status}")
                
        except Exception as e:
            print(f"{keyword:<15} | {'오류':>8} | {'0':>8} | {'0.00':>10} | ⚠️ 실패")
    
    print_subsection("검색 통계")
    print(f"📊 총 검색 키워드: {search_stats['total_searched']}개")
    print(f"📊 총 검색된 문서: {search_stats['total_results']}개")
    print(f"📊 유효 문서: {len(all_docs)}개")
    print(f"📊 키워드 성공률: {(search_stats['found_docs']/max(search_stats['total_searched'], 1)*100):.1f}%")
    
    # 점수 기준 정렬 및 중복 제거
    all_docs.sort(key=lambda x: x['score'], reverse=True)
    
    unique_docs = []
    seen_previews = set()
    
    for doc in all_docs:
        # 문서 내용으로 중복 체크 (앞의 200자)
        content_key = ' '.join(doc['chunk'][:200].replace('\n', ' ').split())
        if content_key not in seen_previews:
            unique_docs.append(doc)
            seen_previews.add(content_key)
            if len(unique_docs) >= max_final_docs:
                break
    
    print_subsection(f"최종 선별 문서 ({len(unique_docs)}개)")
    print(f"{'순위':<4} | {'키워드':<12} | {'검색점수':<10} | {'미리보기'}")
    print("-" * 80)
    
    for i, doc in enumerate(unique_docs, 1):
        clean_preview = ' '.join(doc['preview'].replace('\n', ' ').split())[:50]
        print(f"{i:<4} | {doc['keyword']:<12} | {doc['score']:>10.2f} | {clean_preview}...")
    
    return unique_docs

def find_documents_from_query(query, top_k=10, max_final_docs=10):
    """
    쿼리에서 키워드를 자동 추출하고 문서 검색하는 통합 함수
    
    Args:
        query (str): 사용자 입력 쿼리
        top_k (int): 키워드당 검색할 문서 수
        max_final_docs (int): 최종 선별할 문서 수
        
    Returns:
        tuple: (검색된 문서들, 키워드 정보)
    """
    print_header("쿼리 기반 자동 문서 검색 시스템")
    print(f"📝 입력 쿼리: {query}")
    print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 1단계: AI 키워드 추출
    print_section("1️⃣ AI 키워드 추출")
    keywords, keyword_info = extract_keywords_from_query(query, top_k, max_final_docs)
    
    if not keywords:
        print("❌ 키워드 추출에 실패했습니다.")
        return [], {}
    
    # 2단계: 키워드로 문서 검색
    print_section("2️⃣ 키워드 기반 문서 검색")
    documents = search_documents_with_keywords(keywords, top_k, max_final_docs)
    
    return documents, keyword_info

def create_context_from_best_docs(docs):
    """문서에서 컨텍스트 생성"""
    if not docs:
        return "관련 문서를 찾을 수 없습니다."
    
    print_subsection("컨텍스트 생성")
    
    context_parts = []
    total_length = 0
    
    for i, doc in enumerate(docs, 1):
        doc_content = f"=== 참고문서 {i} (키워드: {doc['keyword']}, 점수: {doc['score']:.2f}) ===\n{doc['chunk']}"
        context_parts.append(doc_content)
        total_length += len(doc['chunk'])
    
    context = "\n\n".join(context_parts)
    
    print(f"📄 총 문서 수: {len(docs)}개")
    print(f"📄 총 컨텍스트 길이: {len(context):,}자")
    print(f"📄 평균 문서 길이: {total_length//len(docs):,}자")
    
    return context

def smart_synopsis_generation(query, context):
    """스마트 시놉시스 생성"""
    print_subsection("AI 기반 내용 분석")
    
    # 1단계: 관련성 평가
    print("🔍 1단계: 자료 관련성 평가 중...")
    relevance_check_prompt = f"""
질문: {query}
제공된 자료: {context[:1500]}...

이 자료가 질문과 얼마나 관련이 있는지 1-10점으로 평가하고, 
주요 관련 내용을 3-5개 불릿포인트로 요약해주세요.

평가 형식:
점수: X/10
주요 내용:
- 내용1
- 내용2
- 내용3
"""
    
    try:
        relevance_response = openai_client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,
            max_tokens=400,
            temperature=0.1,
            messages=[
                {"role": "system", "content": "자료 관련성 평가 전문가입니다."},
                {"role": "user", "content": relevance_check_prompt}
            ]
        )
        
        relevance_info = relevance_response.choices[0].message.content.strip()
        print("✅ 관련성 평가 완료")
        
        # 점수 추출
        relevance_score = "정보없음"
        for line in relevance_info.split('\n'):
            if '점수:' in line:
                relevance_score = line.replace('점수:', '').strip()
                break
        
        print(f"📊 관련성 점수: {relevance_score}")
        
    except Exception as e:
        print(f"⚠️ 관련성 평가 오류: {e}")
        relevance_info = "평가 실패"
        relevance_score = "오류"
    
    # 2단계: 시놉시스 생성
    print("\n📝 2단계: 시놉시스 생성 중...")
    synopsis_prompt = f"""
질문: {query}

관련성 평가: {relevance_info}

참고 자료:
{context}

위 자료를 바탕으로 웹툰/드라마 스타일의 흥미진진한 시놉시스를 작성해주세요!

스타일 요구사항:
1. 400-500자 분량
2. 웹툰이나 사극 드라마처럼 흥미진진하고 역동적으로 작성
3. "~다!", "~었다!", "하지만!", "그런데!" 같은 감탄부호 적극 활용
4. 인물들의 감정과 갈등을 생생하게 묘사
5. 긴장감과 반전이 있는 스토리텔링
6. 독자가 "다음이 궁금해!"라고 느낄 수 있게 작성
7. 구체적인 인물명과 사건을 포함하되 드라마틱하게 각색

예시 톤:
"왕좌를 둘러싼 치열한 권력투쟁이 시작된다! 영조는 외쳤다 - '이 나라에 붕당은 필요 없다!' 하지만 신하들의 속마음은 달랐다..."

시놉시스:
"""
    
    try:
        synopsis_response = openai_client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,
            max_tokens=1000,
            temperature=1.0,
            messages=[
                {
                    "role": "system", 
                    "content": "역사와 이야기를 생생하고 몰입감 있게 전달하는 한국사 교육 전문가입니다."
                },
                {
                    "role": "user", 
                    "content": synopsis_prompt
                }
            ]
        )
        
        synopsis = synopsis_response.choices[0].message.content.strip()
        print("✅ 시놉시스 생성 완료")
        print(f"📊 생성된 시놉시스 길이: {len(synopsis)}자")
        
        return synopsis, relevance_info, relevance_score
        
    except Exception as e:
        print(f"⚠️ 시놉시스 생성 오류: {e}")
        return "시놉시스 생성에 실패했습니다.", relevance_info, relevance_score

def final_evaluation(synopsis, query):
    """최종 평가"""
    print_subsection("품질 평가")
    print("🔍 AI 품질 평가 진행 중...")
    
    eval_prompt = f"""
질문: {query}
작성된 시놉시스: {synopsis}

다음 기준으로 평가해주세요:

1. 질문 적합성 (1-5점): 질문의 핵심을 얼마나 잘 다뤘는가?
2. 내용 구체성 (1-5점): 구체적인 사실과 예시가 포함되었는가?
3. 교육적 가치 (1-5점): 학습에 도움이 되는가?
4. 완성도 (1-5점): 전체적으로 완성도가 높은가?

평가 형식:
1. 질문 적합성: X/5점 - 설명
2. 내용 구체성: X/5점 - 설명  
3. 교육적 가치: X/5점 - 설명
4. 완성도: X/5점 - 설명

총점: XX/20점

장점:
- 장점1
- 장점2

개선점:
- 개선점1
- 개선점2
"""
    
    try:
        eval_response = openai_client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,
            max_tokens=600,
            temperature=0.5,
            messages=[
                {"role": "system", "content": "교육 콘텐츠 평가 전문가입니다."},
                {"role": "user", "content": eval_prompt}
            ]
        )
        
        evaluation = eval_response.choices[0].message.content.strip()
        print("✅ 품질 평가 완료")
        
        # 총점 추출
        total_score = "정보없음"
        for line in evaluation.split('\n'):
            if '총점:' in line:
                total_score = line.replace('총점:', '').strip()
                break
        
        print(f"📊 최종 점수: {total_score}")
        
        return evaluation, total_score
        
    except Exception as e:
        print(f"⚠️ 품질 평가 오류: {e}")
        return "평가에 실패했습니다.", "오류"

def display_final_results(result):
    """최종 결과 출력"""
    print_header("최종 분석 결과", width=100)
    
    # 기본 정보
    print_subsection("🎯 분석 개요")
    print(f"📝 분석 질의: {result['query']}")
    print(f"⏰ 분석 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 추출된 키워드: {len(result.get('keywords', []))}개")
    print(f"📊 사용된 문서: {len(result['relevant_docs'])}개")
    print(f"📊 관련성 점수: {result.get('relevance_score', '정보없음')}")
    print(f"📊 최종 품질 점수: {result.get('total_score', '정보없음')}")
    
    # 키워드 정보
    if result.get('keyword_info'):
        print_subsection("🔑 추출된 키워드 정보")
        kw_info = result['keyword_info']
        if kw_info.get('core'):
            print(f"핵심키워드: {', '.join(kw_info['core'][:5])}")
        if kw_info.get('extended'):
            print(f"확장키워드: {', '.join(kw_info['extended'][:5])}")
        if kw_info.get('persons'):
            print(f"인물키워드: {', '.join(kw_info['persons'][:3])}")
    
    # 검색된 문서 요약
    print_subsection("📚 주요 발견 문서")
    for i, doc in enumerate(result['relevant_docs'][:3], 1):
        clean_content = ' '.join(doc['preview'].replace('\n', ' ').split())
        print(f"{i}. 키워드: [{doc['keyword']}] | 검색점수: {doc['score']:.2f}")
        print(f"   📄 내용: {clean_content}")
        print()
    
    # 생성된 시놉시스
    print_subsection("📖 생성된 시놉시스")
    print("╔" + "═" * 78 + "╗")
    synopsis_lines = result['synopsis'].split('\n')
    for line in synopsis_lines:
        if len(line) > 76:
            words = line.split(' ')
            current_line = ""
            for word in words:
                if len(current_line + word) <= 76:
                    current_line += word + " "
                else:
                    print(f"║ {current_line.ljust(76)} ║")
                    current_line = word + " "
            if current_line.strip():
                print(f"║ {current_line.strip().ljust(76)} ║")
        else:
            print(f"║ {line.ljust(76)} ║")
    print("╚" + "═" * 78 + "╝")
    
    # 품질 평가
    print_subsection("📊 품질 평가 결과")
    eval_lines = result['evaluation'].split('\n')
    for line in eval_lines:
        if line.strip():
            if any(keyword in line for keyword in ['질문 적합성', '내용 구체성', '교육적 가치', '완성도']):
                print(f"  📈 {line}")
            elif '총점:' in line:
                print(f"  🎯 {line}")
            elif line.startswith('장점:') or line.startswith('개선점:'):
                print(f"  📌 {line}")
            elif line.startswith('- '):
                print(f"    {line}")
            else:
                print(f"  {line}")

def ultimate_rag_solution(query, top_k=10, max_final_docs=10):
    """
    쿼리 기반 자동 RAG 시스템 - AI가 키워드를 자동 추출
    
    Args:
        query (str): 사용자 입력 쿼리 (어떤 주제든 가능)
        top_k (int): 키워드당 검색할 문서 수
        max_final_docs (int): 최종 선별할 문서 수
    
    Returns:
        dict: 분석 결과
    """
    print_header(f"AI 자동 키워드 추출 RAG 시스템", width=100)
    print(f"🚀 시스템 시작: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📝 분석 주제: {query}")
    print(f"🎯 검색 설정: 키워드당 {top_k}개, 최종 {max_final_docs}개 문서")
    
    # 1단계: AI 키워드 추출 및 문서 검색
    print_section("1️⃣ AI 키워드 추출 및 문서 검색")
    relevant_docs, keyword_info = find_documents_from_query(query, top_k, max_final_docs)
    
    if not relevant_docs:
        print("❌ 관련 문서를 찾을 수 없습니다.")
        return None
    
    # 추출된 키워드 정보
    keywords = []
    if keyword_info:
        keywords.extend(keyword_info.get('core', []))
        keywords.extend(keyword_info.get('extended', []))
        keywords.extend(keyword_info.get('persons', []))
    
    # 2단계: 컨텍스트 생성
    print_section("2️⃣ 컨텍스트 생성 단계")
    context = create_context_from_best_docs(relevant_docs)
    
    # 3단계: 시놉시스 생성
    print_section("3️⃣ 시놉시스 생성 단계")
    synopsis, relevance_info, relevance_score = smart_synopsis_generation(query, context)
    
    # 4단계: 품질 평가
    print_section("4️⃣ 품질 평가 단계")
    evaluation, total_score = final_evaluation(synopsis, query)
    
    # 결과 정리
    result = {
        'query': query,
        'keywords': keywords,
        'keyword_info': keyword_info,
        'search_settings': {
            'top_k': top_k,
            'max_final_docs': max_final_docs
        },
        'relevant_docs': relevant_docs,
        'context': context,
        'synopsis': synopsis,
        'relevance_info': relevance_info,
        'relevance_score': relevance_score,
        'evaluation': evaluation,
        'total_score': total_score,
        'timestamp': datetime.now()
    }
    
    # 5단계: 최종 결과 출력
    print_section("5️⃣ 최종 결과")
    display_final_results(result)
    
    print_header("분석 완료", width=100)
    print(f"🏁 분석 종료: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    return result

# 사용 예시
if __name__ == "__main__":
    query = "조선시대 왕과 신하 갈등"
    result = ultimate_rag_solution(query)